In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
import glob
import os
from multiprocessing import Process
import sys
sys.path.append('..')
from modules import utils, constants

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Invalid MIT-MAGIC-COOKIE-1 key/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [2]:
SEED = constants.SEED
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
tf.set_random_seed(SEED)
tf.compat.v1.set_random_seed(SEED)
SEED

42

In [3]:
constants.BETA

3

In [4]:
def run_dqn_model(model_type, steps):
    dir_name = f'seed_{SEED}_{steps}'
    parent_dir = f'../models/logs/{model_type}/noisiness/0.3/biopsy_3'
    path = os.path.join(parent_dir, dir_name)
    os.mkdir(path)
  
    if model_type=='dqn':
        model = utils.stable_vanilla_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dqn', filename=f'dqn_{steps}')
    elif model_type=='ddqn':
        model = utils.stable_double_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='ddqn', filename=f'ddqn_{steps}')
    elif model_type== 'dueling_dqn':
        model = utils.stable_dueling_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_dqn', filename=f'dueling_dqn_{steps}')
    elif model_type == 'dueling_ddqn':
        model = utils.stable_dueling_ddqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_ddqn', filename=f'dueling_ddqn_{steps}')
    elif model_type =='dqn_per':
        model = utils.stable_vanilla_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dqn_per', filename=f'dqn_per_{steps}', per=True)
    elif model_type == 'ddqn_per':
        model = utils.stable_double_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='ddqn_per', filename=f'ddqn_per_{steps}', per=True)
    elif model_type == 'dueling_dqn_per':
        model = utils.stable_dueling_dqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_dqn_per', filename=f'dueling_dqn_per_{steps}', per=True)
    elif model_type == 'dueling_ddqn_per':
        model = utils.stable_dueling_ddqn(X_train, y_train, steps, save=True, log_path=path, log_prefix='dueling_ddqn_per', filename=f'dueling_ddqn_per_{steps}', per=True)
    else:
        raise ValueError(f'Unknown model type - {model_type}!')
    return model

In [5]:
train_df = pd.read_csv('../new_data/train_set_noisiness_0.3.csv')
train_df = train_df.fillna(-1)
train_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,proteinuria,biopsy_proven_lupus_nephritis,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
1,1,0,1,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,1,0,0,1,0,...,1,3,0,0,0,0,0,0,0,0


In [6]:
train_df.label.value_counts()

1    25330
0    25070
Name: label, dtype: int64

In [7]:
train_df.iloc[90]

ana                              0
fever                            0
leukopenia                       0
thrombocytopenia                 0
auto_immune_hemolysis            0
delirium                         0
psychosis                        0
seizure                          0
non_scarring_alopecia            0
oral_ulcers                      0
cutaneous_lupus                  0
pleural_effusion                 0
pericardial_effusion             0
acute_pericarditis               0
joint_involvement                0
proteinuria                      0
biopsy_proven_lupus_nephritis    0
anti_cardioliphin_antibodies     0
anti_β2gp1_antibodies            0
lupus_anti_coagulant             0
low_c3                           1
low_c4                           0
anti_dsdna_antibody              0
anti_smith_antibody              0
label                            1
Name: 90, dtype: int64

In [8]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]
X_train, y_train = np.array(X_train), np.array(y_train)
X_train.shape, y_train.shape

((50400, 24), (50400,))

In [9]:
# model_names = ['dqn', 'ddqn', 'dueling_dqn', 'dueling_ddqn', 'dqn_per', 'ddqn_per', 'dueling_dqn_per', 
#                'dueling_ddqn_per']
model_names = ['dueling_dqn_per', 'dueling_ddqn_per']
procs = []
steps = int(10e7)

In [10]:
# run_dqn_model(model_names[0],steps)

In [11]:
for name in model_names:
#     run_dqn_model(name, steps)
    proc = Process(target=run_dqn_model, args=(name, steps))
    procs.append(proc)
    proc.start()

The environment seed is [42]
The environment seed is [42]WARNING:tensorflow:From /home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/stable_baselines/common/tf_util.py:191: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.









Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where












Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










--------------------------------------
| % time spent exploring  | 95       |
| episodes                | 100000   |
| mean 100 episode reward | -1       |
| steps                   | 482677   |
| success rate            | 0.09     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 95       |
| episodes                | 100000   |
| mean 100 episode reward | -0.9     |
| steps                   | 482520   |
| success rate            | 0.13     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 90       |
| episodes                | 200000   |
| mean 100 episode reward | -0.6     |
| steps                   | 980209   |
| success rate            | 0.26     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 90       |
| episodes                | 200000   |
| mean 100 episode reward | -0.7     |
| steps              

--------------------------------------
| % time spent exploring  | 22       |
| episodes                | 1600000  |
| mean 100 episode reward | -0.5     |
| steps                   | 8157899  |
| success rate            | 0.36     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 18       |
| episodes                | 1700000  |
| mean 100 episode reward | 0        |
| steps                   | 8601204  |
| success rate            | 0.54     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 16       |
| episodes                | 1300000  |
| mean 100 episode reward | -1       |
| steps                   | 8809173  |
| success rate            | 0.2      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 14       |
| episodes                | 1800000  |
| mean 100 episode reward | 0        |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2000000  |
| mean 100 episode reward | -0.6     |
| steps                   | 20196108 |
| success rate            | 0.43     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4000000  |
| mean 100 episode reward | 0.3      |
| steps                   | 19872731 |
| success rate            | 0.67     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4100000  |
| mean 100 episode reward | 0.2      |
| steps                   | 20402156 |
| success rate            | 0.64     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4200000  |
| mean 100 episode reward | 0.4      |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6200000  |
| mean 100 episode reward | -0.1     |
| steps                   | 32045696 |
| success rate            | 0.57     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6300000  |
| mean 100 episode reward | 0.2      |
| steps                   | 32666024 |
| success rate            | 0.65     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2900000  |
| mean 100 episode reward | -0.4     |
| steps                   | 33566622 |
| success rate            | 0.49     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 6400000  |
| mean 100 episode reward | 0.3      |
| steps                  

In [ ]:
for proc in procs:
    proc.join()
print('All jobs completed and terminated successfully')